In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [2]:
import pandas as pd
import textstat
import spacy
import nltk
import re
import string
import pandas as pd
import numpy as np
from stop_word_list import *
from stop_word_list_empty import *
from tqdm import tqdm
import sys  
import contractions
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker

# Import data

In [3]:
rawdata = pd.read_csv('D:\Projects\Jupyter\Github Docs\datasets\steam_reviews_dataset.csv', encoding='utf-8')
rawdata

,app_id,app_name,review_text,review_score,review_votes
0,10,Counter-Strike,Ruined my life.,1,0
1,10,Counter-Strike,This will be more of a ''my experience with th...,1,1
2,10,Counter-Strike,This game saved my virginity.,1,0
3,10,Counter-Strike,• Do you like original games? • Do you like ga...,1,0
4,10,Counter-Strike,"Easy to learn, hard to master.",1,1
...,...,...,...,...,...
6417101,99910,Puzzle Pirates,I really ove this game but it needs somethings...,-1,0
6417102,99910,Puzzle Pirates,"Used to play Puzzel Pirates 'way back when', b...",-1,0
6417103,99910,Puzzle Pirates,"This game was aright, though a bit annoying. W...",-1,0
6417104,99910,Puzzle Pirates,"I had a nice review to recommend this game, bu...",-1,0


In [4]:
nmsdata=rawdata[rawdata.app_id == 275850]
nmsdata

,app_id,app_name,review_text,review_score,review_votes
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1
3357655,275850,No Man's Sky,"If they keep updating it, I'll keep playing it.",1,1
3357656,275850,No Man's Sky,"Look, this game received a ton of flak. Howeve...",1,1
3357657,275850,No Man's Sky,The 1.1 update is amazing. If you still have t...,1,1
3357658,275850,No Man's Sky,This game is 100% exploration. Hardly any comb...,1,1
...,...,...,...,...,...
3382364,275850,No Man's Sky,"Tried to get a refund and Steam said no, so I'...",-1,0
3382365,275850,No Man's Sky,"No man's sky is a terrible game, repetitive ga...",-1,1
3382366,275850,No Man's Sky,Lying Devs promised a billion more things and ...,-1,0
3382367,275850,No Man's Sky,This game could ve been much more.. Now its ju...,-1,0


## Sentences

In [5]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
def sent_tokenize_wrapper(text):
    return sent_tokenize(text)


nmsdata['sent_content'] = nmsdata['review_text'].map(lambda x:sent_tokenize(x))
nmsdata = nmsdata.explode('sent_content')
nmsdata

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\researcher\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\2217759078.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nmsdata['sent_content'] = nmsdata['review_text'].map(lambda x:sent_tokenize(x))


,app_id,app_name,review_text,review_score,review_votes,sent_content
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,This was my review prior to the 1.1 update ___...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,"When the game first came out it had over 200,0..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,If that doesn't prove this game is the biggest...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,The devs completely lied about countless featu...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,______________________________________________...
...,...,...,...,...,...,...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,Only about 3 or 4 hours of it was me actually ...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,The free roam exploration runs dry very quickl...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,"Overall, No Man's Sky is a lazy game not even ..."
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,The game's cycle is essentially go to a planet...


In [6]:
nmsdata=nmsdata.dropna()
nmsdata

,app_id,app_name,review_text,review_score,review_votes,sent_content
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,This was my review prior to the 1.1 update ___...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,"When the game first came out it had over 200,0..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,If that doesn't prove this game is the biggest...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,The devs completely lied about countless featu...
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,1,1,______________________________________________...
...,...,...,...,...,...,...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,Only about 3 or 4 hours of it was me actually ...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,The free roam exploration runs dry very quickl...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,"Overall, No Man's Sky is a lazy game not even ..."
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",-1,0,The game's cycle is essentially go to a planet...


# DPP

## Textstat

In [7]:
nmsdata['Automated Readability Index'] = nmsdata['sent_content'].apply(textstat.automated_readability_index)
nmsdata['The Flesch Reading Ease formula'] = nmsdata['sent_content'].apply(textstat.flesch_reading_ease)
nmsdata['The Flesch-Kincaid Grade Level'] = nmsdata['sent_content'].apply(textstat.flesch_kincaid_grade)
nmsdata['The SMOG Index'] = nmsdata['sent_content'].apply(textstat.smog_index)
nmsdata['The Coleman-Liau Index'] = nmsdata['sent_content'].apply(textstat.coleman_liau_index)
nmsdata['Dale-Chall Readability Score'] = nmsdata['sent_content'].apply(textstat.dale_chall_readability_score)
nmsdata['Linsear Write Formula'] = nmsdata['sent_content'].apply(textstat.linsear_write_formula)
nmsdata['The Fog Scale (Gunning FOG Formula)'] = nmsdata['sent_content'].apply(textstat.gunning_fog)
nmsdata['difficult_words'] = nmsdata['sent_content'].apply(textstat.difficult_words)
nmsdata['Readability Consensus based upon all the above tests'] = nmsdata['sent_content'].apply(textstat.text_standard)

nmsdata.to_csv('./processing_datasets/readability.csv',index=False)

C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\3635887407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nmsdata['Automated Readability Index'] = nmsdata['sent_content'].apply(textstat.automated_readability_index)
C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\3635887407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nmsdata['The Flesch Reading Ease formula'] = nmsdata['sent_content'].apply(textstat.flesch_reading_ease)
C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\3635887

In [8]:
data = pd.DataFrame({'id': nmsdata['app_id'],
                     'name':nmsdata['app_name'],
                   'content': nmsdata['review_text'],
                   'sentence': nmsdata['sent_content'],
                   'review_score': nmsdata['review_score'],
                    'review_votes':nmsdata['review_votes'],
                    'readability':nmsdata['Automated Readability Index']       
                    },
                  columns = ['id','name','content', 'sentence', 'review_score','review_votes','readability'])

data

,id,name,content,sentence,review_score,review_votes,readability
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1
...,...,...,...,...,...,...,...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7


## Functions

In [9]:
sent_0 = data['sentence'].values[10]
print(sent_0)
print("="*50)

sent_100 = data['sentence'].values[100]
print(sent_100)
print("="*50)

sent_1500 = data['sentence'].values[1500]
print(sent_1500)
print("="*50)

The three modes are  Normal - this is what most players who revisit the game will know, it's basically the same as before 1.1 but with the new content such as base building  Survival - Practically the same as normal, but with a very steep learning curve.
I do agree that $60 feels steep for a game “a mile wide and an inch deep,” but I myself personally paid full price as an way of “paying forward” further development and because I was very excited for an exploration game that veered more into hard sci-fi.
I bought the game on day 1 and don't regret it even the slightest bit.


In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()

In [11]:
# remove urls from text python: https://stackoverflow.com/a/40823105/4084039
sent_0 = re.sub(r"http\S+", "", sent_0)

print(sent_0)

The three modes are  Normal - this is what most players who revisit the game will know, it's basically the same as before 1.1 but with the new content such as base building  Survival - Practically the same as normal, but with a very steep learning curve.


In [12]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\t", "will not", phrase)
    phrase = re.sub(r"was\'t", "was not", phrase)
    phrase = re.sub(r"gonna", "going to", phrase)
    phrase = re.sub(r"dont", "do not", phrase)
    phrase = re.sub(r"doesnt ", "does not", phrase)
    phrase = re.sub(r"Doesnt ", "does not", phrase)
    phrase = re.sub(r"Dont", "do not", phrase)
    phrase = re.sub(r"Was\'t", "was not", phrase)
    phrase = re.sub(r"Won\t", "will not", phrase)
    phrase = re.sub(r"Gonna", "going to", phrase)

    phrase = re.sub(r"can't", "can not", phrase)
    phrase = re.sub(r"Can't", "Can not", phrase)

    phrase = re.sub(r"nms", " no mans sky ", phrase)
    phrase = re.sub(r" didnt ", " did not ", phrase)
    phrase = re.sub(r" havent ", " have not ", phrase)
    phrase = re.sub(r" cant ", " can not ", phrase)
    phrase = re.sub(r" couldnt ", " could not ", phrase)
    phrase = re.sub(r" wouldnt ", " would not ", phrase)
    phrase = re.sub(r" thats ", " that is ", phrase)
    phrase = re.sub(r" isnt ", " is not ", phrase)
    phrase = re.sub(r" arent ", " are not ", phrase)
    phrase = re.sub(r" ive ", " i have ", phrase)
    phrase = re.sub(r" iam ", " i am ", phrase)
    phrase = re.sub(r" dev ", " developer ", phrase)
    phrase = re.sub(r" devs ", " developers ", phrase)
    phrase = re.sub(r" thx ", " thanks ", phrase)
    phrase = re.sub(r"thx ", " thanks ", phrase)
    phrase = re.sub(r" vs ", " versus ", phrase)
    phrase = re.sub(r" btw ", " by the way ", phrase)
    phrase = re.sub(r" pc ", " personal computer ", phrase)
    phrase = re.sub(r" ur ", " your are ", phrase)
    phrase = re.sub(r" hrs ", " hours ", phrase)
    phrase = re.sub(r" hr ", " hour ", phrase)
    phrase = re.sub(r" E3 ", " electronic entertainment expo ", phrase)
    phrase = re.sub(r" e3 ", " electronic entertainment expo ", phrase)
    phrase = re.sub(r" e3.", " electronic entertainment expo.", phrase)
    phrase = re.sub(r" e3,", " electronic entertainment expo,", phrase)

    phrase = re.sub(r" nms ", " no mans sky ", phrase)
    phrase = re.sub(r" pve ", " player versus environment ", phrase)
    phrase = re.sub(r" pvp ", " player versus player ", phrase)
    phrase = re.sub(r" mmo ", " massively multiplayer online game ", phrase)
    phrase = re.sub(r" tv ", " television ", phrase)
    phrase = re.sub(r" rpg ", " role-playing video game ", phrase)
    phrase = re.sub(r" wp ", " well played ", phrase)
    phrase = re.sub(r" gg ", " good game ", phrase)
    phrase = re.sub(r" dlc ", " downloadable content ", phrase)
    phrase = re.sub(r" npc ", " non-player character ", phrase)
    phrase = re.sub(r" hg ", " hello games ", phrase)
    phrase = re.sub(r"hg ", " hello games ", phrase)

    phrase = re.sub(r" hrs ", " hours ", phrase)
    phrase = re.sub(r" btw ", " by the way ", phrase)
    phrase = re.sub(r" mph ", " miles per hour ", phrase) 
    phrase = re.sub(r" fov ", " field of view ", phrase) 
    phrase = re.sub(r" rly ", " really ", phrase) 
    phrase = re.sub(r" em ", " them ", phrase) 
    phrase = re.sub(r" imho ", " in my humble opinion ", phrase) 
    phrase = re.sub(r" tl;dr ", " too long; did not read ", phrase) 
    phrase = re.sub(r" tldr ", " too long; did not read ", phrase) 
    phrase = re.sub(r" tl dr ", " too long; did not read ", phrase)
    phrase = re.sub(r" tl:dr ", " too long; did not read ", phrase) 
    phrase = re.sub(r" srsly ", " seriously ", phrase) 

    phrase = re.sub(r" bs ", " bullshit ", phrase) 
    phrase = re.sub(r" tbh ", " to be honest ", phrase)
    phrase = re.sub(r" cuz ", " because ", phrase) 
    phrase = re.sub(r" E:D ", " elite dangerous ", phrase)
    phrase = re.sub(r" e:d ", " elite dangerous ", phrase) 
    phrase = re.sub(r" ad ", " advantages ", phrase) 
    phrase = re.sub(r" kok ", " king of kings ", phrase) 
    phrase = re.sub(r" wmr ", " with much respect ", phrase) 
    phrase = re.sub(r" tfw ", " that feel when ", phrase) 
    phrase = re.sub(r" f2p ", " free to play ", phrase) 
    phrase = re.sub(r" lmb ", " left mouse button ", phrase) 
    phrase = re.sub(r" eve ", " everyone versus everyone ", phrase) 
    phrase = re.sub(r" gui ", " game user interface ", phrase) 
    phrase = re.sub(r" c-mon ", " come on ", phrase) 
    phrase = re.sub(r" c'mon ", " come on ", phrase) 
    phrase = re.sub(r" mp ", " multi player ", phrase) 
    phrase = re.sub(r" thx ", " thanks ", phrase) 
    phrase = re.sub(r" mc ", " minecraft ", phrase) 
    phrase = re.sub(r" ftw ", " for the win ", phrase) 
    phrase = re.sub(r" vr ", " virtual reality ", phrase) 
    phrase = re.sub(r" ui ", " user interface ", phrase) 
    phrase = re.sub(r" VR ", " virtual reality ", phrase) 
    phrase = re.sub(r" UI ", " user interface ", phrase) 
    phrase = re.sub(r" isn't ", " is not ", phrase) 
    phrase = re.sub(r" aren't ", " are not ", phrase) 
    phrase = re.sub(r" wasn't ", " was not ", phrase) 
    phrase = re.sub(r" weren't ", " were not ", phrase) 

    
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [13]:
def to_lower(sent) :
    return sent.lower()

In [14]:
# stopwords= set(["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"])
stopwords= set([])

In [15]:
wordnet_lemmatizer = WordNetLemmatizer()

def lemmatize(text):

    from nltk.corpus import wordnet

    def get_wordnet_pos(word):
        """Map POS tag to first character lemmatize() accepts"""
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}

        return tag_dict.get(tag, wordnet.NOUN)

    lemmatized_word = [wordnet_lemmatizer.lemmatize(word, get_wordnet_pos(word))for sent in nltk.sent_tokenize(text)for word in nltk.word_tokenize(sent)]

    return " ".join(lemmatized_word)

In [16]:
def encode_emojis(s):
    s = re.sub(r'♥+', 'profanity', s)
    s = s.replace(':)', 'smiley')
    s = s.replace(':D', 'laughing')
    s = s.replace(':d', 'laughing')
    s = s.replace(' xD ', 'laughing')
    s = s.replace(' x‑D ', 'laughing')
    s = s.replace(' :"D ', 'tears of happiness')
    s = s.replace(' :O ', 'surprise')
    s = s.replace(':)', 'smiley')
    s = s.replace(':(', 'frowney')
    s = s.replace('<3', 'heart')
    s = s.replace(':/', 'exasperation')
    s = s.replace('10/10', 'very satisfied')
    s = s.replace('5/10', 'satisfied')
    s = s.replace('7/10', 'more than satisfied')
    s = s.replace('2/10', 'unsatisfied')
    s = s.replace('0/10', 'unsatisfied')
    s = s.replace('6/10', 'satisfied')
    s = s.replace('8/10', 'more than satisfied')
    s = s.replace('9/10', 'more than satisfied')
    s = s.replace('0/10', 'unsatisfied')
    s = s.replace('1/10', 'unsatisfied')
    s = s.replace('3/10', 'unsatisfied')
    s = s.replace('4/10', 'partly satisfied') 
    
    return s

# Process

In [17]:
preprocessed_reviews = []
for content in tqdm(data['sentence']):
    content = to_lower (content)
    content = re.sub(r"http\S+", "", content)
    content = re.sub(r'\.(?=[^ \W\d])', '. ', content)
    content = content.replace('?', '? ')
    content = content.replace(')', ') ')
    content = content.replace('f*cking', 'fucking')
    content = content.replace('!', '! ')
    content = content.replace(':', ': ')
    content = decontracted(content)
    content = contractions.fix(content)
    content = ' '.join(e.lower() for e in content.split() if e.lower() not in stopwords)
    preprocessed_reviews.append(content.strip())

100%|████████████████████████████████████████████████████████████████████████| 163847/163847 [00:16<00:00, 9920.69it/s]


In [18]:
# second step
preprocessed_reviews_sec = []
for sentance in tqdm(preprocessed_reviews):
    sentance = to_lower (sentance)
    sentance = lemmatize (sentance)
    sentance = decontracted(sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews_sec.append(sentance.strip())

100%|█████████████████████████████████████████████████████████████████████████| 163847/163847 [22:11<00:00, 123.08it/s]


In [19]:
preprocessed_reviews_thr = []
# tqdm is for printing the status bar
for sentance in tqdm(preprocessed_reviews_sec):
    sentance=sentance.replace(' ca not ', ' can not ')
#     sentance = sentance.replace(' : ', '')
#     sentance = sentance.replace(' ! ', '')
#     sentance = sentance.replace(' , ', '')
#     sentance = sentance.replace(' . ', '')
#     sentance = sentance.replace(' _ ', '')
#     sentance = sentance.replace(' ; ', '')
    sentance = re.sub("\S*\d\S*", " ", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews_thr.append(sentance.strip())

100%|███████████████████████████████████████████████████████████████████████| 163847/163847 [00:03<00:00, 48503.95it/s]


In [20]:
print (preprocessed_reviews_thr[10])
print (preprocessed_reviews_thr[100])
print (preprocessed_reviews_thr[150])
print (preprocessed_reviews_thr[200])

the three mode be normal this be what most player who revisit the game will know it be basically the same a before but with the new content such a base building survival practically the same a normal but with a very steep learn curve
i do agree that feel steep for a game a mile wide and an inch deep but i myself personally paid full price a an way of pay forward far development and because i be very excite for an exploration game that veer more into hard sci fi
it probably should have be at most
this update add sooo much to the game new resource new module a base building system the ability to purchase a freighter and upgrade it several bug fix and more well do guy and thank you for not abandon u


## Special replacements

In [21]:
preprocessed_reviews_nm = []
# tqdm is for printing the status bar
for sentance in tqdm(preprocessed_reviews_thr):
    sentance=sentance.replace(' nm ', ' no mans sky ')
    sentance=sentance.replace(' no man sky ', ' no mans sky ')
    sentance=sentance.replace('no man is sky', ' no mans sky ')
    sentance=sentance.replace('hello game ', ' hello games ')
    sentance=sentance.replace(' hello game ', ' hello games ')
    sentance=sentance.replace(' hello game', ' hello games ')
    sentance=sentance.replace(' hellogame ', ' hello games ')
    sentance=sentance.replace(' hellogames ', ' hello games ')
    sentance=sentance.replace(' hg ', ' hello games ')
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords)
    preprocessed_reviews_nm.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████| 163847/163847 [00:01<00:00, 127682.62it/s]


## Additional stopwords

In [22]:
# stopwords_add= set(['game'])
stopwords_add= set([''])

preprocessed_reviews_game = []
# tqdm is for printing the status bar
for sentance in tqdm(preprocessed_reviews_nm):
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in stopwords_add)
    preprocessed_reviews_game.append(sentance.strip())

100%|██████████████████████████████████████████████████████████████████████| 163847/163847 [00:01<00:00, 160587.43it/s]


## Result

In [23]:
preprocessed_reviews_game

['this be my review prior to the update nothis game have fail miserably i be very annoyed that i be stuck with this garbage in my game library because it be completely terrible and i will most likely never play it again',
 'when the game first come out it have over player on at one point now there be a mear',
 'if that do not prove this game be the big scam of i do not know what do',
 'the developer completely lie about countless feature that be meant to be in this game',
 'i clearly have very strong feel about the game after play for about or so hour but now that hello games s have release the new update i decide i need to revist the game in order to see what sort of improvement be make',
 'obviously the update do not add all the feature we be told about before the game be launch but it do add content that we have all hop to receive and that be awesome',
 'it show that hello games s have start take a step in the right direction in hope to fix their game',
 'i really do enjoy the game 

In [24]:
data['preprocessed_reviews'] = preprocessed_reviews_game

In [25]:
data.to_csv('./processing_datasets/preprocessed.csv',index=False)

# Word token

In [26]:
wordtoken_data=pd.read_csv('./processing_datasets/preprocessed.csv', error_bad_lines=False);
wordtoken_data=wordtoken_data.dropna()
wordtoken_data

C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\3079161141.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  wordtoken_data=pd.read_csv('./processing_datasets/preprocessed.csv', error_bad_lines=False);


,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews
0,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...
1,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...
2,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...
3,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...
4,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...
...,...,...,...,...,...,...,...,...
163842,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3,only about or hour of it be me actually enjoy ...
163843,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1,the free roam exploration run dry very quickly...
163844,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7,overall no man be sky be a lazy game not even ...
163845,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7,the game be cycle be essentially go to a plane...


In [27]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = nltk.collocations.BigramCollocationFinder.from_documents([comment.split() for comment in wordtoken_data.preprocessed_reviews])
# Filter only those that occur at least 2 times
finder.apply_freq_filter(200000)
bigram_scores = finder.score_ngrams(bigram_measures.pmi)

bigram_pmi = pd.DataFrame(bigram_scores,columns= ['bigram', 'pmi'])
#bigram_pmi.columns = ['bigram', 'pmi']
bigram_pmi.sort_values(by='pmi', axis = 0, ascending = False, inplace = True)

def bigram_filter(bigram):
    tag = nltk.pos_tag(bigram)
    if tag[0][1] not in ['JJ', 'NN'] and tag[1][1] not in ['NN']:
        return False
    if bigram[0] in nouns_list or bigram[1] in nouns_list:
        return False
    if 'n' in bigram or 't' in bigram:
        return False
    if 'PRON' in bigram:
        return False
    return True

filtered_bigram = bigram_pmi[bigram_pmi.apply(lambda bigram:\
                                              bigram_filter(bigram['bigram'])\
                                              and bigram.pmi > 1, axis = 1)][:200]

bigrams = [' '.join(x) for x in filtered_bigram.bigram.values if len(x[0]) > 1 or len(x[1]) > 1]
bigrams[:300]

[]

In [28]:
# Concatenate n-grams
def replace_ngram(x):
    for gram in bigrams:
        x = x.replace(gram, '_'.join(gram.split()))
    return x

reviews_w_ngrams = data.copy()
reviews_w_ngrams.Preprocess = reviews_w_ngrams.preprocessed_reviews.map(lambda x: replace_ngram(x))
reviews_w_ngrams = reviews_w_ngrams.Preprocess.map(lambda x: [word for word in x.split()\
                                                 if                                                      
                                                              word not in nouns_list\
                                                              and len(word) > 1])

C:\Users\researcher\AppData\Local\Temp\ipykernel_17412\352258278.py:8: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  reviews_w_ngrams.Preprocess = reviews_w_ngrams.preprocessed_reviews.map(lambda x: replace_ngram(x))


In [29]:
reviews_w_ngrams

3357654    [this, be, my, review, prior, to, the, update,...
3357654    [when, the, game, first, come, out, it, have, ...
3357654    [if, that, do, not, prove, this, game, be, the...
3357654    [the, developer, completely, lie, about, count...
3357654    [clearly, have, very, strong, feel, about, the...
                                 ...                        
3382368    [only, about, or, hour, of, it, be, me, actual...
3382368    [the, free, roam, exploration, run, dry, very,...
3382368    [overall, no, man, be, sky, be, lazy, game, no...
3382368    [the, game, be, cycle, be, essentially, go, to...
3382368    [not, even, consider, all, of, the, content, h...
Name: preprocessed_reviews, Length: 163847, dtype: object

In [30]:
data['wordtoken'] = reviews_w_ngrams

In [31]:
data

,id,name,content,sentence,review_score,review_votes,readability,preprocessed_reviews,wordtoken
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,This was my review prior to the 1.1 update ___...,1,1,17.6,this be my review prior to the update nothis g...,"[this, be, my, review, prior, to, the, update,..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,"When the game first came out it had over 200,0...",1,1,6.5,when the game first come out it have over play...,"[when, the, game, first, come, out, it, have, ..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,If that doesn't prove this game is the biggest...,1,1,5.6,if that do not prove this game be the big scam...,"[if, that, do, not, prove, this, game, be, the..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,The devs completely lied about countless featu...,1,1,8.7,the developer completely lie about countless f...,"[the, developer, completely, lie, about, count..."
3357654,275850,No Man's Sky,This was my review prior to the 1.1 update ___...,______________________________________________...,1,1,29.1,i clearly have very strong feel about the game...,"[clearly, have, very, strong, feel, about, the..."
...,...,...,...,...,...,...,...,...,...
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",Only about 3 or 4 hours of it was me actually ...,-1,0,22.3,only about or hour of it be me actually enjoy ...,"[only, about, or, hour, of, it, be, me, actual..."
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The free roam exploration runs dry very quickl...,-1,0,41.1,the free roam exploration run dry very quickly...,"[the, free, roam, exploration, run, dry, very,..."
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...","Overall, No Man's Sky is a lazy game not even ...",-1,0,9.7,overall no man be sky be a lazy game not even ...,"[overall, no, man, be, sky, be, lazy, game, no..."
3382368,275850,No Man's Sky,"No Man's Sky is a game with many problems, the...",The game's cycle is essentially go to a planet...,-1,0,23.7,the game be cycle be essentially go to a plane...,"[the, game, be, cycle, be, essentially, go, to..."


In [32]:
data.to_csv('./processing_datasets/preprocessed_word.csv',index=False)